# Data Science Exploration

This script will fit models on the Top Hits on Spotify from 2000-2019 
- https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019/data 
- This dataset contains audio statistics of the top 2000 tracks on Spotify from 2000-2019. The data contains about 18 columns each describing the track and it's qualities.


In [2]:
# Import required libaries 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [3]:
# Read in the data 
spotifyData = pd.read_csv('spotify.csv')

# GENERAL LINEAR MODEL
Will fit a GLM to the data to predict the popularity of the song

In [41]:
# Define variables, dropping all categorical features, 80-20 train/test split 
X = spotifyData.drop(columns=['popularity', 'artist', 'song', 'genre', 'explicit'])
y = spotifyData['popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Feature Engineering
poly = PolynomialFeatures(degree=2)  # Example degree, adjust as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Data Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

NameError: name 'PolynomialFeatures' is not defined

In [35]:
# Select the top 8 predictors 
k = 8 
selector = SelectKBest(score_func=f_regression, k=k)

# Fit the selector to your training data and transform both training and test data
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Get the selected feature indices and get the names 
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = X_train.columns[selected_feature_indices]

print("Selected Features:", selected_feature_names)

Selected Features: Index(['duration_ms', 'energy', 'loudness', 'mode', 'acousticness',
       'instrumentalness', 'liveness', 'tempo'],
      dtype='object')


In [36]:
# Fit the GLM
model = LinearRegression()
model.fit(X_train_selected, y_train)

LinearRegression()

In [38]:
# Make predictions on the test set
y_pred = model.predict(X_test_selected)

In [39]:
# Evaluate the model performance 
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 472.53880770559755
R-squared: 0.013528820821501086


# RANDOM FOREST 
Will fit a Random Forest model to the data to predict the genre of the song 

In [ ]:
# Select relevant features and target variable
X2 = spotifyData.drop(columns=['artist', 'song', 'genre'])
y2 = spotifyData['genre']

# Encode categorical target variable (artist)
label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(y2)

# Split the data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=41)

In [ ]:
# Initialize and train the model (e.g., Random Forest Classifier)
model = RandomForestClassifier()
model.fit(X2_train, y2_train)

# Make predictions on the testing set
y2_pred = model.predict(X2_test)

In [ ]:
# Evaluate the model using accuracy, precision and recall 
accuracy = accuracy_score(y2_test, y2_pred)
precision = precision_score(y2_test, y2_pred, average='weighted', zero_division=0)  # Weighted average precision
recall = recall_score(y2_test, y2_pred, average='weighted', zero_division=0)  # Weighted average recall

# Print the metrics 
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
# Input my own data to test the model 
example_input = {
    'duration_ms': 228700,
    'explicit': 1,  
    'year': 2003, 
    'popularity': 66,
    'danceability': 0.3,
    'energy': 0.6,
    'key': 3,
    'loudness': 3,
    'mode': 3.5,
    'speechiness': 0.9,
    'acousticness': 0.3, 
    'instrumentalness': 0.3, 
    'liveness': 0.2,
    'valence': 0.3,
    'tempo': 35
}

example_df = pd.DataFrame([example_input])
predicted_genre = model.predict(example_df)
predicted_genre_names = label_encoder.inverse_transform(predicted_genre)
print("Predicted Genre:", predicted_genre_names)